# Implementation of Simulated Annealing

## Initial Model
We will start with 0-Rule model, which is a simple baseline classification model. It does not use any features for prediction but instead predicts the most frequent class (mode) in the training dataset. This can serve as a baseline to compare the performance of more sophisticated models.

## Temparature
We use the temperature to escape local maximums, we sometimes accept bad values but we gradually decrease the frequency of accepting bad values.

if eval(vc ) < eval(vn ) then vc = vn -> possibility of accepting a state with a better outcome is always 1.
else calculate the acceptance probability is  np.exp((new_score - current_score) / temperature)

In [15]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

class AutoML:
    def __init__(self, initial_temp=100, cooling_rate=0.99, max_iterations=100, min_training_time=3600):
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.min_training_time = min_training_time

        self.algorithms = {
            'DecisionTreeClassifier': {
                'class': DecisionTreeClassifier,
                'parameters': ['max_depth', 'min_samples_split'],
                'ranges': [(1, 20), (2, 20)]
            },
            'SVC': {
                'class': SVC,
                'parameters': ['C', 'gamma'],
                'ranges': [(0.01, 10), (0.01, 1)]
            },
            'RandomForestClassifier': {
                'class': RandomForestClassifier,
                'parameters': ['n_estimators', 'max_depth'],
                'ranges': [(10, 100), (1, 20)]
            },
            'GradientBoostingClassifier': {
                'class': GradientBoostingClassifier,
                'parameters': ['n_estimators', 'learning_rate'],
                'ranges': [(10, 100), (0.01, 0.3)]
            },
            'MLPClassifier': {
                'class': MLPClassifier,
                'parameters': ['alpha', 'learning_rate_init'],
                'ranges': [(0.0001, 0.1), (0.0001, 0.1)]
            },
            'GaussianNB': { 
                'class': GaussianNB,
                'parameters': [],  
                'ranges': []      
            }
        }

        self.best_solution = None
        self.best_score = 0
        self.model = None

    def eval(self, model, X, y):
        model.fit(X, y) 
        predictions = model.predict(X)  
        accuracy = np.mean(predictions == y)  
        print(f'Accuracy: {accuracy:.4f}')
        return accuracy
        
    def generate_neighborhood_withsmallchanges(self, current_solution):
        algorithm_name = current_solution[0]
        algorithm_info = self.algorithms[algorithm_name]
        new_solution = current_solution[:]
    
        if not algorithm_info['parameters']:
            new_solution[0] = np.random.choice(list(self.algorithms.keys()))
            return new_solution
    
        while len(new_solution) < len(algorithm_info['parameters']) + 1:
            new_solution.append(None)
    
        param_idx = np.random.randint(1, len(new_solution))
        low, high = algorithm_info['ranges'][param_idx - 1]
    
        if isinstance(low, int) and isinstance(high, int):
            new_solution[param_idx] = np.random.randint(low, high)
        elif isinstance(low, float) and isinstance(high, float):
            new_solution[param_idx] = np.random.uniform(low, high)
    
        if np.random.rand() < 0.1:
            new_solution[0] = np.random.choice(list(self.algorithms.keys()))
            algorithm_info = self.algorithms[new_solution[0]]
            new_solution = [new_solution[0]] + [
                np.random.uniform(low, high) if isinstance(low, float) and isinstance(high, float)
                else np.random.randint(low, high)
                for low, high in algorithm_info['ranges']
            ]
    
        print(f"Generated neighborhood for algorithm: {new_solution[0]}, parameters: {new_solution[1:]}")
        return new_solution


        
    def generate_neighborhood(self, current_solution):
        if not isinstance(current_solution, list):
            current_solution = [current_solution.__class__.__name__]
    
        algorithm_name = np.random.choice(list(self.algorithms.keys()))
        algorithm_info = self.algorithms[algorithm_name]
    
        new_solution = [algorithm_name] + [None] * len(algorithm_info['parameters'])
    
        for i, parameter in enumerate(algorithm_info['parameters']):
            if len(algorithm_info['ranges']) == 0:
                continue 
    
            low, high = algorithm_info['ranges'][i]
    
            if low is not None and high is not None:
                if isinstance(high, list):  
                    new_solution[i + 1] = np.random.choice(high)
                elif isinstance(high, str):  
                    current_idx = algorithm_info['ranges'][i].index(high)
                    new_idx = (current_idx - 1) % len(algorithm_info['ranges'][i])
                    new_solution[i + 1] = algorithm_info['ranges'][i][new_idx]
                elif isinstance(low, int) and isinstance(high, int): 
                    new_solution[i + 1] = np.random.randint(low, high)
                else:  
                    new_solution[i + 1] = np.random.uniform(low, high)
    
        print(f"Neighborhood algorithm: {algorithm_name}, parameters: {new_solution[1:]}")
        return new_solution

    def create_model(self, solution):
        algorithm_name = solution[0]
        hyperparameters = solution[1:]
        algorithm_info = self.algorithms[algorithm_name]
        algorithm_class = algorithm_info['class']
    
        if algorithm_name == 'MLPClassifier':
            alpha = hyperparameters[0] if len(hyperparameters) > 0 else 0.0001
            learning_rate_init = hyperparameters[1] if len(hyperparameters) > 1 else 0.001
            return algorithm_class(alpha=alpha, learning_rate_init=learning_rate_init)
        elif algorithm_name == 'GaussianNB':
            return algorithm_class()
    
        valid_params = {
            param: int(value) if param in ['max_depth', 'n_estimators', 'min_samples_split'] else value
            for param, value in zip(algorithm_info['parameters'], hyperparameters)
            if value is not None
        }
        return algorithm_class(**valid_params)
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.simulated_annealing()

    def predict(self, X):
        if self.model is None:
            raise ValueError("The model has not been fit yet. Please call the fit method first.")
        return self.model.predict(X)

    def simulated_annealing(self):
        start_time = time.time()  
        # 0 rule model as initial model as base model
        zero_r_model = DummyClassifier(strategy='most_frequent')
        #zero_r_model.fit(self.X, self.y) 

        print(f"Initial model: DummyClassifier")
        print(f"Initial parameters: (strategy='most_frequent')")

        current_solution = ['DummyClassifier']
        current_score = self.eval(zero_r_model, self.X, self.y)
        best_solution = current_solution
        best_score = current_score
    
        temperature = self.initial_temp
    
        while time.time() - start_time < self.min_training_time:
            for i in range(100):
                if i % 10 == 0:
                    print(f"Iteration {i}, Temperature {temperature:.3f}, Best Evaluation {best_score:.5f}")
                    
                if current_solution[0] == 'DummyClassifier':
                    new_solution = self.generate_neighborhood(['DecisionTreeClassifier'])
                    new_score = self.eval(self.create_model(new_solution), self.X, self.y)
                else:
                    new_solution = self.generate_neighborhood_withsmallchanges(current_solution)
                    new_score = self.eval(self.create_model(new_solution), self.X, self.y)
    
                if new_score > current_score:
                    current_solution = new_solution
                    current_score = new_score
                    if new_score > best_score:
                        best_solution = new_solution
                        best_score = new_score
                else:
                    acceptance_probability = np.exp((new_score - current_score) / temperature)
                    if np.random.rand() < acceptance_probability:
                        current_solution = new_solution
                        current_score = new_score
    
            temperature *= self.cooling_rate
    
        self.best_solution = best_solution
        self.best_score = best_score
        self.model = self.create_model(best_solution)
        self.model.fit(self.X, self.y)
        print(f'best_score is {best_score}')
        print(f'best_solution is {best_solution}')
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)

In [16]:
column_names = ["Sex", "Length", "Diameter", "Height", "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight", "Rings"]
df_abalone = pd.read_csv("./data/abalone.csv", header=0, names=column_names)
df_abalone[df_abalone.Height == 0]
df_abalone = df_abalone[df_abalone.Height != 0]
df_abalone = pd.get_dummies(df_abalone, columns=['Sex'], drop_first=False)

def categorize_rings(rings):
    if rings < 8:
        return 0
    elif rings <= 12:
        return 1
    else:
        return 2

df_abalone['Rings_Category'] = df_abalone['Rings'].apply(categorize_rings)

display(df_abalone.head())
display(df_abalone.info())
X = df_abalone.drop("Rings", axis=1).values
y = df_abalone["Rings"].values

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M,Rings_Category
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1,2
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1,0
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0,1
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4175 entries, 0 to 4176
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          4175 non-null   float64
 1   Diameter        4175 non-null   float64
 2   Height          4175 non-null   float64
 3   Whole_weight    4175 non-null   float64
 4   Shucked_weight  4175 non-null   float64
 5   Viscera_weight  4175 non-null   float64
 6   Shell_weight    4175 non-null   float64
 7   Rings           4175 non-null   int64  
 8   Sex_F           4175 non-null   uint8  
 9   Sex_I           4175 non-null   uint8  
 10  Sex_M           4175 non-null   uint8  
 11  Rings_Category  4175 non-null   int64  
dtypes: float64(7), int64(2), uint8(3)
memory usage: 338.4 KB


None

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

automl = AutoML(min_training_time=10) 
automl.fit(X_train, y_train)
predictions = automl.predict(X_test)

Initial model: DummyClassifier
Initial parameters: (strategy='most_frequent')
Accuracy: 0.1667
Iteration 0, Temperature 100.000, Best Evaluation 0.16672
Neighborhood algorithm: DecisionTreeClassifier, parameters: [1, 15]
Accuracy: 0.2558
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 15]
Accuracy: 0.4449
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 15]
Accuracy: 0.4449
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 12]
Accuracy: 0.4462
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 16]
Accuracy: 0.4449
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 4]
Accuracy: 0.4481
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 4]
Accuracy: 0.4481
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 2]
Accuracy: 0.4484
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters

/Users/sibeltoprakkiran/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy: 0.3782
Generated neighborhood for algorithm: MLPClassifier, parameters: [0.04930467555610184, 0.030871487367649397]
Accuracy: 0.3906
Generated neighborhood for algorithm: MLPClassifier, parameters: [0.08452080378847067, 0.030871487367649397]
Accuracy: 0.3858
Iteration 60, Temperature 100.000, Best Evaluation 0.95145
Generated neighborhood for algorithm: GaussianNB, parameters: []
Accuracy: 0.3229
Accuracy: 1.0000
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [4, None]
Accuracy: 0.3996
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [4, 8]
Accuracy: 0.3996
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [6, 8]
Accuracy: 0.4478
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [19, 8]
Accuracy: 0.7752
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [19, 15]
Accuracy: 0.6765
Generated neighborhood for algorithm: DecisionTreeClassifier, parameters: [7,

KeyboardInterrupt: 